In [1]:
from splinter import Browser
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re

In [11]:
zipcodes = pd.read_csv('C:/Anaconda/US_zipcodes.csv')
# zipcodes.zip_code[0:5]
len(zipcodes.zip_code)

41855

In [8]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': 'C:/Anaconda/webscrapes/chromedriver'}
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('log-level=3')
browser = Browser('chrome', **executable_path, wait_time=1,headless=True,options=chrome_options)


In [9]:
# Visit Carrier Dealer Locator site
url = 'https://www.carrier.com/residential/en/us/find-a-dealer/'
browser.visit(url)



In [10]:
# This script will scrape dealer information based on a list of zipcodes
# The dealer information is contained in a javascript array of objects 



df=pd.DataFrame()
for z in zipcodes.zip_code[0:25]:
    browser.cookies.delete()
    browser.find_by_name(name="ctl00$pageContent$FindADealer$dealerLocator").fill(z)
    browser.find_by_id("pageContent_FindADealer_btnSubmit").click()
    html = browser.html
    carrier_soup = BeautifulSoup(html, 'html.parser')
    dict_list=[]
    for i in range(29)[19:]:
        try:  #finds and extracts only portion of javascript embedded in html that contains array of objects(python dictionary) as a string
            dict_list.append(re.findall("(?<=\\')[^\\[),';].*?(?=\\')",carrier_soup.find_all('script')[i].get_text(strip=True)))  
        
        except:
            pass
#this block cleans up list of objects to be converted in list of dictionaries then to a dataframe
    clean_list=[]   
    for d in dict_list:
        try:
            clean_list.append(eval(d[0]))
        except:
            pass
    for d in clean_list:  #this dealer no is the list number on each page not the actual dealer account number
        del d['DealerNo']
        
    df=df.append(pd.DataFrame(clean_list),ignore_index=True)
    df.drop_duplicates(inplace=True)

browser.quit()

# df.drop('DealerNo',axis=1)

df['Country']='United States'
df['Brand'] = 'Carrier'
df['DealerName']= df['DealerName'].apply(lambda x: x.replace('amp;',''))
df['DealerName']= df['DealerName'].apply(lambda x: x.replace("#39;",''))
d2 = pd.DataFrame([df.DealerName,df.PostalAddress,df.City,df.State,df.ZipCode,df.Country,df.Phone,df.LocatorLatitude,df.LocatorLongitude,df.Website,df.Brand]).T



from sqlalchemy import create_engine
import urllib


params = urllib.parse.quote_plus('DRIVER={SQL Server Native Client 10.0};Server=gmlsql2hou; DATABASE=invplannersapp;Trusted_Connection=yes;')

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
d2.to_sql('GBU_Webscrape',engine,if_exists='append',index=False)


# print("--- %s seconds ---" % (time.time() - start_time))

In [20]:
df.drop('DealerNo',axis=1)
# dict_list

,City,DealerName,LocatorLatitude,LocatorLongitude,Phone,PostalAddress,State,Website,ZipCode
0,Anchorage,Circle Plumbing &amp;amp; Heating,61.1518000000001,-149.9469,888-581-3991,2317 Raspberry Rd,AK,http://circleplumbingandheating.com/,99502
1,Wasilla,Trans Alaska Mechanical LLC,61.5950000000001,-149.3769,888-493-8668,1255 W Winter Ct,AK,http://www.trans-ak.com,99654
2,Wasilla,Trans Alaska Mechanical LLC,61.5950000000001,-149.3769,888-493-8668,1255 W Winter Ct,AK,http://www.trans-ak.com,99654


In [6]:
df['Country']='United States'
df['Brand'] = 'Carrier'
df['DealerName']= df['DealerName'].apply(lambda x: x.replace('amp;',''))
df['DealerName']= df['DealerName'].apply(lambda x: x.replace("#39;",''))
d2 = pd.DataFrame([df.DealerName,df.PostalAddress,df.City,df.State,df.ZipCode,df.Country,df.Phone,df.LocatorLatitude,df.LocatorLongitude,df.Website,df.Brand]).T
# d2

,DealerName,PostalAddress,City,State,ZipCode,Country,Phone,LocatorLatitude,LocatorLongitude,Website,Brand
0,Circle Plumbing & Heating,2317 Raspberry Rd,Anchorage,AK,99502,United States,888-581-3991,61.1518000000001,-149.9469,http://circleplumbingandheating.com/,Carrier
1,Trans Alaska Mechanical LLC,1255 W Winter Ct,Wasilla,AK,99654,United States,888-493-8668,61.5950000000001,-149.3769,http://www.trans-ak.com,Carrier


In [85]:
# d2.to_csv('carrier_scrape.csv',index=False)

In [1]:
from sqlalchemy import create_engine
import urllib


params = urllib.parse.quote_plus('DRIVER={SQL Server Native Client 10.0};Server=gmlsql2hou; DATABASE=invplannersapp;Trusted_Connection=yes;')

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
d2.to_sql('GBU_Webscrape',engine,if_exists='append',index=False)


NameError: name 'd2' is not defined

In [ ]:
# cn = pyodbc.connect('DRIVER={SQL Server Native Client 10.0};Server=gmlsql2hou; DATABASE=invplannersapp;Trusted_Connection=yes;')

# dftest = pd.read_sql('select * from invplannersapp.dbo.gbu_webscrape',cn)
# dftest

In [6]:
start = [x for x in range(0,50000,1000)]
finish = [x for x in range(1000,51000,1000)]

In [8]:
len(finish)

50